In [1]:
import re

import spacy

from funer.document import Document
from funer.annotators.dictionary_annotator import DictionaryAnnotator
from funer.annotators.token_condition_annotator import (
    TokensConditionAnnotator, generate_token_conditions_function)
from funer.annotators.span_condition_annotator import SpanConditionAnnotator
from funer.document import Document
from funer.labeling_function_applier import LabelingFunctionApplier
from funer.aggregators.majority_voting_aggregators import MajorityVotingAggregator
from funer.utils import show_labels

## Documentの作成

### 分かち書き済みのテキストを使う

In [2]:
labeled_document_1 = Document(
    tokens=["東京", "出身", "の", "吉田", "は", "4", "月", "から", "JR", "で", "働く", "。"],
    gold_label=["B-LOC", "O", "O", "B-PER", "O", "B-DATE", "I-DATE", "O", "B-ORG", "O", "O", "O"]
)
labeled_document_2 = Document(
    tokens=["9", "月", "から", "東京", "大学", "に", "通う"],
    gold_label=["B-DATE", "I-DATE", "O", "B-ORG", "I-ORG", "O", "O"],
)
nolabeled_document = Document(
    tokens=["8", "月", "に", "東京", "の", "大学", "に", "通う"],
)
documents = [labeled_document_1, labeled_document_2, nolabeled_document]

### Option: spaCyを使う

In [3]:
nlp = spacy.load('ja_ginza')

In [4]:
document = Document.from_spacy_doc(nlp("東京出身の吉田は4月からJRで働く。"))
print(document.tokens)

['東京', '出身', 'の', '吉田', 'は', '4', '月', 'から', 'JR', 'で', '働く', '。']


## ラベリング関数

### ラベリング関数の定義

In [5]:
# f1: トークン単位でのラベリング関数
def detect_name(tokens):
    for i in range(len(tokens) - 1):
        if tokens[i:i + 1] == ["吉田"]:
            yield i, i + 1
f1 = TokensConditionAnnotator(
    name="person_f",
    f=detect_name,
    label="PER"
)

# f2: generate_token_conditions_functionを用いた、トークン単位でのラベリング関数
f2 = TokensConditionAnnotator(
    name="month_f",
    f=generate_token_conditions_function([
        lambda token_1: re.search(r"[1-9]|1[0-2]", token_1) is not None,
        lambda token_2: token_2 == "月",
    ]),
    label="DATE"
)

# f3: 文字単位でのラベリング関数
def span_condition_function(text: str):
    for m in re.finditer(r"JR", text):
        yield m.start(), m.end()
f3 = SpanConditionAnnotator(
    name="company_f",
    f=span_condition_function,
    label="ORG"
)


# f4: 辞書を用いたラベリング関数
#   : (注意) 東京大学[LOC]の東京をORGとして間違って抽出する例
loc_dictionary = ["東京", "神奈川", "大阪"]
f4 = DictionaryAnnotator(
    name="pref_f",
    words=loc_dictionary,
    label="LOC"
)

### ラベリング関数の適用と統合

In [6]:
# ラベリング関数の適用
lf_applier = LabelingFunctionApplier(lfs=[f1, f2, f3, f4])
documents = lf_applier.apply(documents)

# ラベリング結果の統合
aggregator = MajorityVotingAggregator()
documents = aggregator.aggregate(documents)

# 結果の出力
print(show_labels(documents[0]))
print()
print(show_labels(documents[1]))
print()
print(show_labels(documents[2]))

tokens       東京    出身   の   吉田    は   4        月       から   JR      で   働く   。
gold_label   B-LOC   O      O    B-PER   O    B-DATE   I-DATE   O      B-ORG   O    O      O 
---------------------------------------------------------------------------------------------
person_f     O       O      O    B-PER   O    O        O        O      O       O    O      O 
month_f      O       O      O    O       O    B-DATE   I-DATE   O      O       O    O      O 
company_f    O       O      O    O       O    O        O        O      B-ORG   O    O      O 
pref_f       B-LOC   O      O    O       O    O        O        O      O       O    O      O 
---------------------------------------------------------------------------------------------
aggregate    B-LOC   O      O    B-PER   O    B-DATE   I-DATE   O      B-ORG   O    O      O 

tokens       9        月       から   東京    大学    に   通う
gold_label   B-DATE   I-DATE   O      B-ORG   I-ORG   O    O   
--------------------------------------------------

In [7]:
print(lf_applier.show_stats())

f_name    | pos | neg | hit
==========+=====+=====+====
person_f  | 1   | 0   | 1  
month_f   | 2   | 0   | 2  
company_f | 1   | 0   | 1  
pref_f    | 1   | 1   | 2  


In [8]:
print(documents[2].export_span_labels())

[EntitySpan(start_offset=0, end_offset=2, label='DATE'), EntitySpan(start_offset=3, end_offset=5, label='LOC')]


In [9]:
print(documents[2].export_bio_label())

['B-DATE', 'I-DATE', 'O', 'B-LOC', 'O', 'O', 'O', 'O']
